In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
import numpy as np
from matplotlib import pyplot as plt

import sys
sys.path.append('../dependencies/')
import dataset_utils
import network
import utils
import zca

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

tf.random.set_seed(42)

tfd = tfp.distributions
tfk = tf.keras
tfkl = tf.keras.layers

train_set = 'compcars'
norm = None
mode = 'color'

if mode == 'color':
    input_shape = (32, 32, 3)
    datasets = [
        'svhn_cropped',
        'cifar10',
        'celeb_a',
        'gtsrb',
        'compcars',
        'noise'
    ]
    num_filters = 64
elif mode == 'grayscale':
    input_shape = (32, 32, 1)
    datasets = [
    'mnist',
    'fashion_mnist',
    'emnist/letters',
    'sign_lang',
    'noise'
    ]
    num_filters = 32

if norm is None:
    dir_str = 'original'
elif norm == 'pctile-5':
    dir_str = 'pctile-5'
elif norm == 'channelwhiten':
    dir_str = 'zca'
elif norm == 'zca_original':
    dir_str = 'zca_original'
elif norm == 'histeq':
    dir_str = 'histeq'

In [2]:
import tqdm
from scipy.io import savemat
from scipy.io import loadmat
import cv2

probs = {}

if train_set == 'emnist/letters':
    train_set1 = 'emnist_letters'
else:
    train_set1 = train_set
    
probs1 = loadmat('../probs/lrat/' + dir_str + '/' + train_set1 + '.mat')

def find_png_size(x):
    x = x.astype('int32')
    tmp = []
    for i in range (x.shape[0]):
        tmp.append(len(cv2.imencode('.png', np.squeeze(x[i,:,:,:]), [int(cv2.IMWRITE_PNG_COMPRESSION), 9])[1])*8)
    return tmp

def find_flif_size(x):
    x = x.astype('int32')
    tmp = []
    for i in range (x.shape[0]):
        cv2.imwrite('/home/barath/temp.png', np.squeeze(x[i,:,:,:]))
        !flif -e -o /home/barath/temp.png /home/barath/temp.flif &> out.log
        tmp.append(os.path.getsize('/home/barath/temp.flif')*8)
    return tmp


save_dir = '../probs/' + 'ic_flif/' + dir_str + '/'
if not os.path.exists(save_dir+train_set1+'.mat'):
    
    for dataset in datasets:

        _, _, ds_test = dataset_utils.get_dataset(
              dataset,
              256,
              mode,
              normalize=norm,
              dequantize=False,
              visible_dist='categorical',
              mutation_rate=0
          )
        tmp = []
        for test_batch in tqdm.tqdm(ds_test):
            tmp.append(find_flif_size(test_batch.numpy()))

        tmp = np.expand_dims(np.concatenate(tmp, axis=0),axis=-1)
        probs[dataset+'_size'] = tmp
        probs[dataset] = -(probs1[dataset]/np.log(2)) - tmp
        
else:
    probs = loadmat(save_dir+train_set1+'.mat')
    
    for dataset in datasets:
        probs[dataset] = -(probs1[dataset]/np.log(2)) - probs[dataset+'_size']

save_dir = '../probs/' + 'ic_flif/' + dir_str + '/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

savemat(save_dir + train_set1 + '.mat', probs)
print('Done')

Done


In [3]:
# import tensorflow as tf
# import tensorflow_datasets as tfds
# import tensorflow_probability as tfp
# import numpy as np
# from matplotlib import pyplot as plt

# import sys
# sys.path.append('../kushal_code/')
# import dataset_utils
# import utils
# import zca
# import tqdm
# from scipy.io import savemat
# from scipy.io import loadmat
# import cv2

# import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"]="1"
# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)

# norm = None
# mode = 'color'

# if mode == 'color':
#     input_shape = (32, 32, 3)
#     datasets = [
#         'svhn_cropped',
#         'cifar10',
#         'celeb_a',
#         'gtsrb',
#         'compcars',
#     ]
#     num_filters = 64
# elif mode == 'grayscale':
#     input_shape = (32, 32, 1)
#     datasets = [
#     'mnist',
#     'fashion_mnist',
#     'emnist/letters',
#     'sign_lang'
#     ]
#     num_filters = 32

# if norm is None:
#     dir_str = 'original'
# elif norm == 'pctile-5':
#     dir_str = 'pctile-5'
# elif norm == 'channelwhiten':
#     dir_str = 'zca'
# elif norm == 'zca_original':
#     dir_str = 'zca_original'
# elif norm == 'histeq':
#     dir_str = 'histeq'
    
# def find_flif_size(x):
#     x = x.astype('int32')
#     tmp = []
#     for i in range (x.shape[0]):
#         cv2.imwrite('/home/barath/temp.png', np.squeeze(x[i,:,:,:]))
#         !flif -e -o /home/barath/temp.png /home/barath/temp.flif &> out.log
#         tmp.append(os.path.getsize('/home/barath/temp.flif')*8)
#     return tmp

# _, _, ds_test = dataset_utils.get_dataset(
#               'noise',
#               256,
#               mode,
#               normalize=norm,
#               dequantize=False,
#               visible_dist='categorical',
#               mutation_rate=0
#           )
# tmp = []
# for test_batch in tqdm.tqdm(ds_test):
#     tmp.append(find_flif_size(test_batch.numpy()))

# tmp = np.expand_dims(np.concatenate(tmp, axis=0),axis=-1)

# for dataset in datasets:
#     if dataset == 'emnist/letters':
#         dataset = 'emnist_letters'
#     probs = loadmat('../probs/' + 'ic_flif/' + dir_str + '/' + dataset + '.mat')
#     probs['noise_size'] = tmp
#     savemat('../probs/' + 'ic_flif/' + dir_str + '/' + dataset + '.mat', probs)